Les données sont dispo à cette url :

http://alt.qcri.org/semeval2016/task5/index.php?id=data-and-tools

In [ ]:
!pip install pandas_read_xml

In [62]:
import pandas_read_xml as pdx
import numpy as np
from functools import partial

In [63]:
df = pdx.read_xml(
    "datasets/ABSA16FR_Restaurants_Train-withcontent.xml",
    ['Reviews']
)

Les Données contiennent plus d'information que nécessaire : chaque commentaire est séparé en phrase qui ont elles même plusieurs notes chacune (service, nourriture, prix ...). Pour simplifier le problème, on va prendre la moyenne des notes et garder une note finale :
- `'negative'` (0)
- `'neutral'` (1)
- `'positive'` (2)

In [64]:
def extract_text(row):
    
    sent = row['sentences']['sentence']
    
    if isinstance(sent, list):
        text = '. '.join([s['text'] for s in sent if s['text'] is not None])
    else:
        text = sent['text']

    return text

In [65]:
def extract_label(row, is_binary=True):
    
    sent = row['sentences']['sentence']
    
    def _extract_label(r, is_binary=True):
        
        if is_binary:
            correspondance_labels = {
                'negative' : 0,
                'neutral' : .5,
                'positive' : 1
            }
        else:
            correspondance_labels = {
                'negative' : 0,
                'neutral' : 1,
                'positive' : 2
            }
        
        opinions = r['Opinions']['Opinion']
        
        if isinstance(opinions, list):
            polarities = [o['@polarity'] for o in opinions]
            pol_int = [correspondance_labels[p] for p in polarities]

            label = int(np.round(np.mean(pol_int)))
        
        else:
            label = correspondance_labels[opinions['@polarity']]
            
        return label
    
    if isinstance(sent, list):
        labels = [x for x in sent if x is not None]
        labels = [x for x in labels if x['Opinions'] is not None]
        labels = [_extract_label(s, is_binary) for s in labels]
        label = int(np.round(np.mean(labels)))
    else:
        label = _extract_label(sent, is_binary)
    
    return label

In [66]:
df['texte'] = df.Review.apply(extract_text)
df['label_2'] = df.Review.apply(partial(extract_label, is_binary=True))
df['label_3'] = df.Review.apply(partial(extract_label, is_binary=False))

In [67]:
df.texte[100]

'Service de tres bonne qualitÃ©, restaurant agreable et cuisine delicieuse !. Les produits sont frais et de saison, cest un regal !. Jy retournerai a coup sur pour decouvrir les autres plats de la carte.'

In [68]:
df.label_2.value_counts()

0    187
1    148
Name: label_2, dtype: int64

In [69]:
df.label_3.value_counts()

2    138
0    138
1     59
Name: label_3, dtype: int64

In [71]:
df.drop('Review', axis=1).to_csv("datasets/restaurants.csv")
df.drop('Review', axis=1).to_csv("restaurants.csv")

In [59]:
import pandas as pd

df = pd.read_csv("restaurants.csv", index_col=0)

In [60]:
df.label_3.value_counts()

0    187
1    148
Name: label_3, dtype: int64